In [12]:
import matplotlib.pyplot as plt
from cifnet.scm_synthetic import *
from cifnet.dissimilarities import *
from cifnet.data_utils import *
from cifnet.networks import EmbeddingNet, SiameseNet

# Generate Samples

In [22]:
data_name = "lin"               # data_name: data name includes compas, adult, lin, nlm, imf, loan
data_type = "contrastive"       # data_type: contrastive, triplet
num_points = 1000               # num_points: number of points to generate
margin = 0.1                    # margin: margin for triplet loss or diameter for distance loss
Q_metric = l_p                  # Q_metric: semi-latent metric
hidden_dim = 1000
output_type = "embedding"       # output_type: type of the output, either label or embedding
epochs = 10
method='spearman'
decorrelation_loss_fn = "xicor" # decorrelation_loss_fn: type of the decorrelation loss function, either spearman or xicor of None

In [23]:
scm = get_scm(data_name)
len(scm.mean)
input_dim = len(scm.mean)
embedding_dim = len(scm.mean) - len(scm.get_sensitive())

# Load Data

In [24]:
data_dict = learning_data(data_name, data_type, num_points, margin, Q_metric)
train_dict, test_dict = dict_to_train_test(data_dict)

# Choose The Model

In [25]:
embedding_net = EmbeddingNet(input_dim, embedding_dim, hidden_dim)
model = SiameseNet(embedding_net, Q_metric, margin)

In [26]:
from cifnet.trainers import Trainer
trainer = Trainer()

In [27]:
trainer.train(model, data_dict, data_type, output_type, epochs, decorrelation_loss_fn, margin, Q_metric)

AssertionError: 

In [41]:
x = train_dict['X1']
x = x.detach().numpy()
n, k = x.shape
m = np.zeros((k, k), np.float32)

In [42]:
m

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

In [38]:
x.dtype

dtype('float32')

In [30]:
import os
import pandas as pd
import numpy as np
path='../results/'
def result_to_DF(path='../results/', save ='../plots/'):
    files = os.listdir(path)
    df_cols = ["data", "type", "decorrelation", "output_type", "metric", "seed", "margin", "lambda", "indicator"]
    split_columns = [s.replace('.npy', '').split("_") for s in files]
    df = pd.DataFrame(split_columns, columns=df_cols)
    df['value'] = np.nan
    for i in range(len(files)):
            df.loc[i, "value"] = float(np.load(path + files[i]))

    df.to_csv(save + 'results.csv')
    return "File saved to " + save + 'results.csv'

In [31]:
result_to_DF()

C:\Users\ahmad\AppData\Local\Temp\ipykernel_15488\1352670106.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  df.loc[i, "value"] = float(np.load(path + files[i]))


'File saved to ../plots/results.csv'

In [ ]:
def result_to_DF(path='results/'):
    df_cols = ['dataset', 'trainer', 'model', 'lambd', 'seed', 'epsilon']
    indicators = ['accs', 'mccs', 'uai_05', 'uai_01', 'uai_cf', 'uai_ar_05', 'uai_ar_01']

    def indicator_to_DF(path, indicator):
        all_files = os.listdir(path)
        files = [f for f in all_files if indicator in f]
        split_columns = [s.replace(indicator, '').replace('.npy', '').split("_") for s in files]
        df = pd.DataFrame(split_columns, columns=df_cols).dropna()
        df[indicator] = np.nan
        for i in range(len(files)):
            df.loc[i, indicator] = float(np.load(path + files[i]))
        return df

    merged_df = indicator_to_DF(path, indicators[0])
    for ind in indicators[1:]:
        merged_df = merged_df.merge(indicator_to_DF(path, ind), how='outer', on=df_cols)

    merged_df.drop(columns=['lambd', 'epsilon'], inplace=True)
    merged_df['trainer'] = merged_df['trainer'].str.split("-").str.get(0)
    merged_df['seed'] = merged_df['seed'].str.replace("s", "")

    return merged_df